In [2]:
def sigma_z_rectangle(x,y,z,s0,a,b):
    xi = np.array([-a/2,-a/2,a/2,a/2])
    yi = np.array([-b/2,b/2,b/2,-b/2])
    Ri = np.sqrt( (x+xi)**2 + (y+yi)**2 + z**2 )
    
    term1i = ( 1/((x+xi)**2 + z**2) + 1/((y+yi)**2 + z**2) ) * ( (x+xi)*(y+yi)*z ) / Ri
    term2i = np.arctan( ( (x+xi)*(y+yi) ) / (Ri * z) )
    signi = np.array([(-1)**i for i in range(0,4)])
    
    sigmai = signi*(term1i + term2i)
    return s0/(2 * np.pi) * sigmai.sum()

In [2]:
def plot_isobars_zz():
    a = 1
    b = 0.4
    fig, ax = plt.subplots(ncols=2,figsize=(24,10))
    xs = np.linspace(-1.2*a,1.2*a,200)
    zs = np.linspace(0.001,2*a,100)
    sigmas = np.zeros((len(xs),len(zs)))
    for i,x in enumerate(xs):
        for j,z in enumerate(zs):
            sigmas[i,j] = sigma_z_rectangle(x,0.0,z,1,a,b)
    X, Z = np.meshgrid(xs,zs)
    cp = ax[0].contour(X, -Z, sigmas.T,levels=10)
    ax[0].clabel(cp, inline=True, 
              fontsize=12)
    ax[0].plot([-a/2,a/2],[0,0],lw=10,color='black')
    ax[0].set_xlabel('$x$ / $a$')
    ax[0].set_ylabel('$z$ / $a$')
    ax[0].set_yticks([0,-1,-2])
    ax[0].set_yticklabels([0,1,2])
    ax[0].set(adjustable='box', aspect='equal')
    #
    ys = np.linspace(-1.2*a,1.2*a,200)
    zs = np.linspace(0.001,2*a,100)
    sigmas = np.zeros((len(ys),len(zs)))
    for i,y in enumerate(ys):
        for j,z in enumerate(zs):
            sigmas[i,j] = sigma_z_rectangle(0.0,y,z,1,a,b)
    Y, Z = np.meshgrid(ys,zs)
    cp = ax[1].contour(Y, -Z, sigmas.T,levels=10)
    ax[1].clabel(cp, inline=True, 
              fontsize=12)
    ax[1].plot([-b/2,b/2],[0,0],lw=10,color='black')
    ax[1].set_xlabel('$y$ / $a$')
    ax[1].set_ylabel('$z$ / $a$')
    ax[1].set_yticks([0,-1,-2])
    ax[1].set_yticklabels([0,1,2])
    ax[1].set(adjustable='box', aspect='equal')
    fig.tight_layout();

In [2]:
def i_Rechteck(z,a,b):
    R = np.sqrt(a**2 + b**2 + z**2)
    term1 = b*z/(b**2+z**2) * a*(R**2 + z**2)/((a**2+z**2)*R)
    term2 = np.arctan( b/z * (a*(a**2+b**2) - 2*a*z * (R-z))/((a**2+b**2)*(R-z) - z*(R-z)**2) )
    return (term1+term2)/(2*np.pi)

In [1]:
from ipywidgets import widgets
from ipywidgets import interact
def interactive_i_rechteck():
    @interact(#c=widgets.BoundedFloatText(value=0.3,min=0, description='$c$ / MPa'),
               z_to_b=widgets.BoundedFloatText(value=1, min=0.01, max=20., description='$z/b$'),
               a_to_b=widgets.BoundedFloatText(value=2, min=1, max=10., description='$a/b$')
    )
    def plot_MC_states(z_to_b,a_to_b):
        plt.rcParams['axes.spines.right'] = True
        fig, ax = plt.subplots(figsize=(10,10))
        sns.set_palette("Paired",n_colors=5)
        ax2 = ax.twinx()
        z1 = np.linspace(0.001,20,100)
        z2 = np.linspace(0.001,2,100)
        for a in [1,2,3,5,10]:
            ax.plot(i_Rechteck(z1,a,1),-z1,label='%.1f' %a)
            ax2.plot(i_Rechteck(z2,a,1),-z2)
        
        i = i_Rechteck(z_to_b,a_to_b,1)
        ax.set_xlim(0,0.25)
        ax.set_ylim(top=0.)
        ax2.set_ylim(top=0.)
        ax.grid(which='both')
        #ax.set_xticks([0.001,0.002,0.0063,0.02,0.063,0.2,0.63,2.0,6.3,20,63])
        #ax.set_xticklabels([0.001,0.002,0.0063,0.02,0.063,0.2,0.63,2.0,6.3,20,63])
        if (z_to_b < 2):
            ax2.plot(i_Rechteck(z2,a_to_b,1),-z2,label='%.1f' %a_to_b,color='black',ls='--')
            ax2.plot([0.25,i,i],[-z_to_b,-z_to_b,0],color='black',ls=':')
            ax2.plot(i,-z_to_b,marker='o',color='red')
        else: 
            ax.plot(i_Rechteck(z1,a_to_b,1),-z1,label='%.1f' %a_to_b,color='black',ls='--')
            ax.plot([0,i,i],[-z_to_b,-z_to_b,0],color='black',ls=':')
            ax.plot(i,-z_to_b,marker='o',color='red')
        fig.legend(loc='upper center',ncol=3)
        plt.show();